### Data Engineer Assignment

### Task
Your colleagues from data science team have developed a prediction model with the following scripts in this notebook, now they need you to help them to <b> put this model in production </b>. In specific, the model should run on a new set of subscribers every Tuesday at 8am, and its output will be delivered to stakeholders and used in the campaigns.


### Sub-tasks
1. Create a weekly scheduled job that uses the model to predict whether a subscriber with purchase a product02, and that delivers the output to stakeholders. You can schedule the job with any tools that you prefer. 
<br>

2. Rewrite/restructure the existing codes if necessary, the purpose is to make the codes cleaner and easy to maintain.
<br>

3. Use CI/CD pipeline to deploy the model, meaning you use CI/CD pipeline to create the weekly job instead of a manual process. This would make us very easy to make changes and continuously improve the model.
<br>

4. Use MLFlow to keep track of the model.

### Hints
1. These tools can help you schedule your weekly job: crontab, airflow, or something on AWS, make sure that the job runs on a dockerized container
2. CI/CD tools: drone, Jenkins



### Load py libaries

In [1]:
# The codes have been tested with python 3.7.9

from typing_extensions import Literal
# import all neccessary libraries
from datetime import datetime
from datetime import date

import numpy as np
import pandas as pd
import xgboost as xgb

from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from functools import partial
from hyperopt import fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import confusion_matrix, roc_auc_score

from hyperopt import hp

import matplotlib.pyplot as plt
#import shap

### Load the data

In [2]:
# load the data
data_file = 'telecom.csv'
raw_data = pd.read_csv(data_file)

In [3]:
raw_data.head()

,subscriber,income,age,var1,gender,house_type,lastVisit,product02
0,1,118239.80,74.32310,1.216512,f,OWNER,1/1/2012 2:09,Nee
1,2,348760.40,45.86138,3.672188,f,OWNER,1/1/2012 14:03,Ja
2,3,96111.34,43.18818,0.578956,f,OWNER,1/1/2012 23:58,Nee
3,4,73818.17,43.32941,-0.129441,m,RENTAL,1/2/2012 3:51,Ja
4,5,36469.51,34.87954,-0.127675,m,RENTAL,1/2/2012 13:50,Nee


### Data Preparation

In [4]:
# prepare the data 

df = raw_data.copy()

# fill missing values
df[['gender', 'house_type']]= df[['gender', 'house_type']].fillna(value='missing')
df['income'] = df['income'].fillna((df['income'].mean()))
df['var1'] = df['var1'].fillna((df['var1'].mean()))
df['age'] = df['age'].fillna((df['age'].mean()))

# log transformation
df['income'] = np.log(df['income'])

# prepare target variable
df.loc[df.product02=='Nee', 'product02'] = 0
df.loc[df.product02=='Ja', 'product02'] = 1
df['product02'] = df.product02.astype(int)

In [5]:
# feature engineering
def calculate_duration(lastVisit):
    if lastVisit:
      lastVisit = datetime.strptime(lastVisit, "%m/%d/%Y %H:%M").date()
      today = date.today()
      return (today - lastVisit).days
    else:
      return 0

df['duration'] = df['lastVisit'].apply(calculate_duration)
df = df.drop(['subscriber','lastVisit'], axis=1)


In [6]:
# one-hot encoding the categorical features
df_X = df.loc[:, df.columns != 'product02']
df_X_onehot = pd.get_dummies(df_X, dummy_na=True)
df_onehot = pd.concat([df_X_onehot, df[['product02']]], axis=1)

In [7]:
strat = df['product02']
df_train, df_test = train_test_split(
        df_onehot,
        test_size=0.2,
        random_state=12345,
        stratify=strat)

df_train_X = df_train.loc[:, df_train.columns != 'product02']
df_train_y = df_train['product02']
df_test_X = df_test.loc[:, df_test.columns != 'product02']
df_test_y = df_test['product02']


In [8]:
df

,income,age,var1,gender,house_type,product02,duration
0,11.680470,74.32310,1.216512,f,OWNER,0,4632
1,12.762140,45.86138,3.672188,f,OWNER,1,4632
2,11.473263,43.18818,0.578956,f,OWNER,0,4632
3,11.209360,43.32941,-0.129441,m,RENTAL,1,4631
4,10.504232,34.87954,-0.127675,m,RENTAL,0,4631
...,...,...,...,...,...,...,...
9519,11.281307,29.57617,2.261101,m,OWNER,1,2846
9520,12.279089,46.34125,0.590807,f,OWNER,1,2846
9521,11.296737,37.10520,-0.099423,f,RENTAL,1,2846
9522,11.975464,65.85983,-0.263309,f,RENTAL,1,2846


## Model Training

In [13]:
def score(params):
    print("Training with params: ")
    print(params)
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(df_train_X, label=df_train_y)
    dvalid = xgb.DMatrix(df_test_X, label=df_test_y)
    watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    gbm_model = xgb.train(params, dtrain, num_round,
                          evals=watchlist,
                          verbose_eval=True)
    predictions = gbm_model.predict(dvalid,
                                    ntree_limit=gbm_model.best_iteration + 1)
    score = roc_auc_score(df_test_y, predictions)
    # TODO: Add the importance for the selected features
    #print(gbm_model.feature_importances_)
    print("\tScore {0}\n\n".format(score))
    # The score function should return the loss (1-score)
    # since the optimize function looks for the minimum
    loss = 1 - score
    return {'loss': loss, 'status': STATUS_OK}



def optimize(
             trials, 
             random_state=1998):
    """
    This is the optimization function that given a space (space here) of 
    hyperparameters and a scoring function (score here), finds the best hyperparameters.
    """
    # To learn more about XGBoost parameters, head to this page: 
    # https://github.com/dmlc/xgboost/blob/master/doc/parameter.md
    space = {
        'n_estimators': hp.quniform('n_estimators', 100, 1000, 1),
        'eta': hp.quniform('eta', 0.025, 0.5, 0.025),
        # A problem with max_depth casted to float instead of int with
        # the hp.quniform method.
        'max_depth':  hp.choice('max_depth', np.arange(1, 14, dtype=int)),
        'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
        'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
        'gamma': hp.quniform('gamma', 0.5, 1, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
        'eval_metric': 'auc',
        'objective': 'binary:logistic',
        # Increase this number if you have more cores. Otherwise, remove it and it will default 
        # to the maxium number. 
        'nthread': 4,
        'booster': 'gbtree',
        'tree_method': 'exact',
        'silent': 1,
        'seed': random_state
    }
    # Use the fmin function from Hyperopt to find the best hyperparameters
    best = fmin(score, space, algo=tpe.suggest, 
                trials=trials, 
                max_evals=250)
    return best


best_hyperparams = optimize(
                            Trials()
                            )
print("The best hyperparameters are: ", "\n")
print(best_hyperparams)

Training with params:                                  
{'booster': 'gbtree', 'colsample_bytree': 0.9500000000000001, 'eta': 0.5, 'eval_metric': 'auc', 'gamma': 0.6000000000000001, 'max_depth': 3, 'min_child_weight': 2.0, 'n_estimators': 968.0, 'nthread': 4, 'objective': 'binary:logistic', 'seed': 1998, 'silent': 1, 'subsample': 0.6000000000000001, 'tree_method': 'exact'}
[0]	eval-auc:0.65701	train-auc:0.69072                 
[1]	eval-auc:0.67164	train-auc:0.71285                 
[2]	eval-auc:0.67449	train-auc:0.71850                 
[3]	eval-auc:0.67576	train-auc:0.71933                 
[4]	eval-auc:0.68124	train-auc:0.72309                 
[5]	eval-auc:0.68129	train-auc:0.72411                 
[6]	eval-auc:0.68352	train-auc:0.73262                 
[7]	eval-auc:0.68405	train-auc:0.73425                 
[8]	eval-auc:0.68627	train-auc:0.73961                 
[9]	eval-auc:0.68800	train-auc:0.74104                 
[10]	eval-auc:0.68686	train-auc:0.74257                
[11]	eval

/opt/homebrew/Caskroom/miniforge/base/envs/py38/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [14:34:15] WARNING: /var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_d9k8pmaj4_/croot/xgboost-split_1724073758172/work/src/learner.cc:740: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



[39]	eval-auc:0.68204	train-auc:0.78163                
[40]	eval-auc:0.68084	train-auc:0.78339                
[41]	eval-auc:0.68105	train-auc:0.78428                
[42]	eval-auc:0.67968	train-auc:0.78494                
[43]	eval-auc:0.67987	train-auc:0.78542                
[44]	eval-auc:0.68099	train-auc:0.78579                
[45]	eval-auc:0.68407	train-auc:0.78665                
[46]	eval-auc:0.68284	train-auc:0.78787                
[47]	eval-auc:0.68079	train-auc:0.78874                
[48]	eval-auc:0.67988	train-auc:0.78975                
[49]	eval-auc:0.68153	train-auc:0.78910                
[50]	eval-auc:0.68008	train-auc:0.79116                
[51]	eval-auc:0.67903	train-auc:0.79328                
[52]	eval-auc:0.67726	train-auc:0.79514                
[53]	eval-auc:0.67866	train-auc:0.79686                
[54]	eval-auc:0.67959	train-auc:0.79778                
[55]	eval-auc:0.67935	train-auc:0.79877                
[56]	eval-auc:0.67856	train-auc:0.79996         

job exception: `best_iteration` is only defined when early stopping is used.



  0%|          | 0/250 [00:04<?, ?trial/s, best loss=?]


AttributeError: `best_iteration` is only defined when early stopping is used.

In [11]:
# train the best model
best_params = best_hyperparams
best_params.update({'eval_metric': 'auc'})

model = XGBClassifier(silent=True)
model.set_params(**best_params)
dtrain = xgb.DMatrix(df_train_X, label=df_train_y)
dvalid = xgb.DMatrix(df_test_X, label=df_test_y)
watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
model = xgb.train(best_params, dtrain, 
                  evals=watchlist, 
                  verbose_eval=True)


NameError: name 'best_hyperparams' is not defined

### Check model performance on test set

In [10]:
# calculate model performance

test_set_prediction = model.predict(xgb.DMatrix(df_test_X))
foo1 = pd.DataFrame({'predict': test_set_prediction}, index=df_test_X.index)
df_test.loc[:, 'predict'] = foo1['predict']
df_test.sort_values('predict', ascending=False, inplace=True)

# 10% of the popultion is selected for target campaigns
p= 0.1
df_test.loc[:, 'predict_label'] = 0
cutoff_val = df_test.iloc[int(p*len(df_test)), ]['predict']
df_test.loc[df_test.predict>= cutoff_val, 'predict_label'] = 1

NameError: name 'model' is not defined

In [12]:
df_test.predict_label.value_counts().sort_index()

0    1714
1     191
Name: predict_label, dtype: int64

In [13]:
a = confusion_matrix(
    df_test['product02'],
    df_test['predict_label'],
    labels = [0, 1])

print(a)

print('\n\n')
actual_postive = len(df_test.loc[df_test.loc[:, 'product02'] == 1, :])
total_num = len(df_test)
predicted_and_actual_postive = len(df_test.loc[
    (df_test.loc[:, 'product02'] == 1)
    &
    (df_test.loc[:, 'predict_label']), :])
predicted_postive = sum(df_test['predict_label'])
model_lift = (predicted_and_actual_postive / predicted_postive) / (actual_postive / total_num)

print(f"{actual_postive} positives out of total {total_num} customers, which is " +
                    f"{ (100 * (actual_postive) / total_num):.02f} %")
print(f"{predicted_and_actual_postive} actual positives out of total {predicted_postive} " +
                    "predicted positives, which is " +
                    f"{ (100 * (predicted_and_actual_postive) / predicted_postive):.02f} %")
print(f"MODEL LIFT = {model_lift:.03f}")

print(f"Precision: { (100 * predicted_and_actual_postive / predicted_postive):.02f} %")
print(f"Recall   : { (100 * predicted_and_actual_postive / actual_postive):.02f} %")

print(f"By contacting {(100 * predicted_postive / total_num):.02f} % of the base, you hit " +
                    f"{(100 * predicted_and_actual_postive / actual_postive):.02f} % of your target group")

[[1099   25]
 [ 615  166]]



781 positives out of total 1905 customers, which is 41.00 %
166 actual positives out of total 191 predicted positives, which is 86.91 %
MODEL LIFT = 2.120
Precision: 86.91 %
Recall   : 21.25 %
By contacting 10.03 % of the base, you hit 21.25 % of your target group


In [14]:
# save model
model.save_model("best_model_xgb.json")

In [15]:
# save shaply plot for model explanation
explainer = shap.TreeExplainer(model)
shap_values = explainer(df_test_X)
shap.summary_plot(shap_values, df_test_X, show=False)
plt.savefig('shap_plot.png', dpi=200, bbox_inches='tight')
plt.close()

### Use the saved model to do scoring

In [21]:

# load the scoring data
df_test = pd.read_csv('telecom_test.csv')

# Prepare the scoring data
# fill missing values
df_test[['gender', 'house_type']]= df_test[['gender', 'house_type']].fillna(value='missing')
df_test['income'] = df_test['income'].fillna((df_test['income'].mean()))
df_test['var1'] = df_test['var1'].fillna((df_test['var1'].mean()))
df_test['age'] = df_test['age'].fillna((df_test['age'].mean()))

# log transformation
df_test['income'] = np.log(df_test['income'])

# feature engineering
def calculate_duration(lastVisit):
    if lastVisit:
      lastVisit = datetime.strptime(lastVisit, "%m/%d/%Y %H:%M").date()
      today = date.today()
      return (today - lastVisit).days
    else:
      return 0

df_test['duration'] = df_test['lastVisit'].apply(calculate_duration)
df_test = df_test.drop(['lastVisit'], axis=1)

df_test_onehot = pd.get_dummies(df_test, dummy_na=True)
df_test_onehot = df_test_onehot.drop(['subscriber'], axis=1)

# load the saved model and use it to predict
dtest = xgb.DMatrix(df_test_onehot)
model_xgb_pred = xgb.Booster()
model_xgb_pred.load_model("best_model_xgb.json")
print(model_xgb_pred)
scores = model_xgb_pred.predict(dtest)

In [22]:
# save the output in csv
df_test['score'] = scores
output = df_test[['subscriber', 'score']]
output.to_csv('output.csv', index=False)